<a href="https://colab.research.google.com/github/rishi-latchmepersad/TinyML-Home-Weather-Forecasting/blob/main/machine_learning/rnn/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rishi-latchmepersad/TinyML-Home-Weather-Forecasting/blob/main/machine_learning/rnn/model_training.ipynb)

In [ ]:
import pandas as pd
from pathlib import Path
import requests
from datetime import datetime


# Import Data
We need to concatenate all readings from the on-device measurement CSVs and augment the dataset with a year of Open-Meteo historical data for Calcutta #3, Couva, Trinidad (August 2024 through August 2025).

In [ ]:

# Locate measurement CSVs whether running locally or in Colab
candidate_dirs = [
    Path('/content/measurements'),
    Path('measurements'),
    Path('../measurements'),
]
measurements_dir = None
for candidate in candidate_dirs:
    if candidate.exists():
        measurements_dir = candidate
        break
if measurements_dir is None:
    raise FileNotFoundError("Could not find a measurements directory in the expected locations.")

csv_files = sorted(measurements_dir.glob('measurements_*.csv'))
if not csv_files:
    raise FileNotFoundError(f"No measurement CSVs found under {measurements_dir}")

measurement_dfs = []
for csv_file in csv_files:
    df_temp = pd.read_csv(csv_file)
    measurement_dfs.append(df_temp)

measurement_df = pd.concat(measurement_dfs, ignore_index=True)
print(f"Successfully loaded and concatenated {len(csv_files)} CSV files into a single DataFrame.")
print(f"The final measurement DataFrame has {measurement_df.shape[0]} rows and {measurement_df.shape[1]} columns.")
measurement_df.head()


In [ ]:

# Keep only the columns needed for the long-to-wide pivot and coerce values to numeric
measurement_df = measurement_df[["timestamp_iso8601", "quantity", "value"]]
measurement_df["value"] = pd.to_numeric(measurement_df["value"], errors="coerce")
measurement_df = measurement_df.dropna(subset=["timestamp_iso8601", "quantity", "value"])
measurement_df.head()


## Pull one year of Open-Meteo data for Calcutta #3 (Couva, Trinidad)
We fetch August 2024 through August 2025 hourly history so the model can train on both on-device measurements and public weather data.

In [ ]:

openmeteo_params = {
    "latitude": 10.4226,  # Calcutta #3, Couva, Trinidad
    "longitude": -61.4749,
    "start_date": "2024-08-01",
    "end_date": "2025-08-01",
    "hourly": [
        "temperature_2m",
        "relative_humidity_2m",
        "surface_pressure",
        "precipitation",
        "wind_speed_10m",
    ],
    "timezone": "UTC",
}

response = requests.get(
    "https://archive-api.open-meteo.com/v1/archive",
    params={**openmeteo_params, "hourly": ",".join(openmeteo_params["hourly"])},
    timeout=60,
)
response.raise_for_status()
openmeteo_hourly = response.json()["hourly"]

openmeteo_long_records = []
quantity_rename = {
    "temperature_2m": "om_temperature_2m_c",
    "relative_humidity_2m": "om_relative_humidity_pct",
    "surface_pressure": "om_surface_pressure_pa",
    "precipitation": "om_precipitation_mm",
    "wind_speed_10m": "om_wind_speed_10m_ms",
}
for param_key, column_name in quantity_rename.items():
    values = openmeteo_hourly[param_key]
    if param_key == "surface_pressure":
        values = [v * 100 for v in values]
    for ts, value in zip(openmeteo_hourly["time"], values):
        openmeteo_long_records.append(
            {"timestamp_iso8601": ts, "quantity": column_name, "value": value}
        )

openmeteo_df = pd.DataFrame(openmeteo_long_records)
openmeteo_df["timestamp_iso8601"] = pd.to_datetime(
    openmeteo_df["timestamp_iso8601"], utc=True
)
print(
    f"Loaded {len(openmeteo_df)} Open-Meteo rows spanning "
    f"{openmeteo_df['timestamp_iso8601'].min()} to {openmeteo_df['timestamp_iso8601'].max()}."
)
openmeteo_df.head()


In [ ]:

# Combine the device measurements with the Open-Meteo history
combined_df = pd.concat([measurement_df, openmeteo_df], ignore_index=True)
df = combined_df
df["timestamp_iso8601"] = pd.to_datetime(df["timestamp_iso8601"], format='mixed', utc=True)
print(f"Combined dataset has {len(df)} rows from {df['timestamp_iso8601'].min()} to {df['timestamp_iso8601'].max()}")
df.head()


# Data Pre-Processing

## Pivoting - Reformating the data into multiple columns
We make each quantity (measurement type e.g. lux, pressure, temperature) its own column, so each row will have one value per quantity instead of one row per quantity.

In [ ]:
df["quantity"].value_counts()

In [ ]:
df = df.pivot_table(values='value', index='timestamp_iso8601',
                       columns='quantity', aggfunc="mean").reset_index()
print(df.columns.name)
df.head()

In [ ]:
df.columns.name = None

In [ ]:

# Drop rain indicator if present (older datasets sometimes include it)
if "is_raining" in df.columns:
    df = df.drop(["is_raining"], axis=1)
df.head()


In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.info()

## Resampling - Handling small gaps between sensor readings
Each quantity is captured by a different sensor. Since the data is captured every 10 seconds, there may be very slight differences (seconds or milliseconds) between when each sensor actually captures its information. This results in readings that represent the same 10 second period appear as different rows. In reality, they should represent the same instance. **Resampling** organizes the data into 30 minute chunks, and takes the average of each value within that 30 minute chunk, thus consolidating them into a single row.

In [ ]:
df["timestamp_iso8601"] = pd.to_datetime(df["timestamp_iso8601"], format='mixed')
df.info()

In [ ]:
# In the original dataset, the sensor readings may have come in at very slightly
# different times, i.e. 1 second apart. But each of these should have technically
# been one instance. So we resample, to get the average of each value within the
# specified resample time.

df = df.set_index("timestamp_iso8601").resample('30min').mean()
df.head()

In [ ]:
len(df)

In [ ]:
df = df.sort_values("timestamp_iso8601")
df.head()

## Interpolation - Handling gaps in the continuous time data
After resampling, there would have been periods where the board was turned off. This results in gaps in the data. Interpolation uses the nearest values to the missing times to fill them.

In [63]:
df.loc["2025-09-04 22:00:00+00:00":"2025-09-05 00:00:00+00:00"]

,humidity_pct,lux_lx,om_precipitation_mm,om_relative_humidity_pct,om_surface_pressure_pa,om_temperature_2m_c,om_wind_speed_10m_ms,pressure_pa,temperature_c,sine_hour,cos_hour,delta_temperature,temp_mean_6h,humidity_mean_6h,target_temperature_c
timestamp_iso8601,,,,,,,,,,,,,,,
2025-09-04 22:00:00+00:00,-1.849565,-0.655435,0.0,0.0,0.0,7.105427e-15,-3.552714e-15,-0.534381,-0.173247,-0.709987,1.225918,0.000000,-0.205588,-1.910267,28.172483
2025-09-04 22:30:00+00:00,-1.312136,-0.653129,0.0,0.0,0.0,7.105427e-15,-3.552714e-15,-0.558529,-0.130578,-0.709987,1.225918,0.173283,-0.180186,-1.631824,28.347315
2025-09-04 23:00:00+00:00,-0.868645,-0.644958,0.0,0.0,0.0,7.105427e-15,-3.552714e-15,-0.534775,0.414303,-0.368763,1.367140,2.224210,0.044540,-1.385826,30.579943
2025-09-04 23:30:00+00:00,-0.747509,-0.644901,0.0,0.0,0.0,7.105427e-15,-3.552714e-15,-0.325886,0.350705,-0.368763,1.367140,-0.260682,0.137972,-1.231447,30.319356
2025-09-05 00:00:00+00:00,-0.552051,-0.644897,0.0,0.0,0.0,7.105427e-15,-3.552714e-15,-0.219706,0.207997,-0.002585,1.415308,-0.583754,0.160048,-1.098313,29.734615


In [64]:
print(f"Data ranges from {df.index[0]} to {df.index[-1]} and we have {len(df)} instances")

Data ranges from 2025-09-04 22:00:00+00:00 to 2025-10-21 19:00:00+00:00 and we have 2251 instances


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2251 entries, 2025-09-04 22:00:00+00:00 to 2025-10-21 19:00:00+00:00
Freq: 30min
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   humidity_pct              2251 non-null   float64
 1   lux_lx                    2251 non-null   float64
 2   om_precipitation_mm       2251 non-null   float64
 3   om_relative_humidity_pct  2251 non-null   float64
 4   om_surface_pressure_pa    2251 non-null   float64
 5   om_temperature_2m_c       2251 non-null   float64
 6   om_wind_speed_10m_ms      2251 non-null   float64
 7   pressure_pa               2251 non-null   float64
 8   temperature_c             2251 non-null   float64
 9   sine_hour                 2251 non-null   float64
 10  cos_hour                  2251 non-null   float64
 11  delta_temperature         2251 non-null   float64
 12  temp_mean_6h              2251 non-null   float64
 13  hum

In [ ]:
# Plotting
import matplotlib.pyplot as plt

# The last 5-day period
ax = df["humidity_pct"].plot(grid=True, marker='.', figsize=(8, 3.5))

df["temperature_c"].plot(
    ax=ax, color='green', linewidth=2, label="Temperature (C)"
)

# Add a legend to distinguish them
ax.legend()

In [ ]:
# The gaps indicate times the board was off

In [ ]:
df = df.interpolate(method="linear")
df.info()

In [ ]:

import numpy as np

# Remove any remaining non-finite values before scaling to avoid NaNs in training weights
numeric_cols = df.select_dtypes(include="number").columns
before_clean = len(df)
df = df.loc[:, ~df.columns.duplicated()]
df = df[np.isfinite(df[numeric_cols]).all(axis=1)]
print(f"Dropped {before_clean - len(df)} rows with non-finite values.")
df.head()


In [ ]:
# The last 5-day period
ax = df["humidity_pct"].plot(grid=True, marker='.', figsize=(8, 3.5))

df["temperature_c"].plot(
    ax=ax, color='green', linewidth=2, label="Temperature (C)"
)
# Add a legend to distinguish them
ax.legend()

# Feature Engineering
We include the following features:
1. `sine_hour` - This captures the cyclical nature of the hour of the day. e.g. when taking the sine of the hour of the day, 23:00 is closer to 00:00 as it should be.
2. `cos_hour` - Similar to the above.
3. `temperature_delta` -  The difference between the temperature at the current timestamp and the timestamp immediately before.
4. `temp_mean_6h` - The average temperature over the past 6 hours at this current timestamp.
5. `temp_humidity_6h` - The average humidity over the past 6 hours at this current timestamp.

In [ ]:
hour_of_day = df.index.hour
hour_of_day

In [ ]:
import numpy as np

df["sine_hour"] = np.sin(2 * np.pi * hour_of_day / 24)
df["cos_hour"] = np.cos(2 * np.pi * hour_of_day / 24)
df.head()

In [ ]:
df["delta_temperature"] = df["temperature_c"] - df["temperature_c"].shift(1)
df.head()

In [ ]:
# Since the first row will be null, we fill it with the mean for this column
df['delta_temperature'].fillna(df['delta_temperature'].mean(), inplace=True)
df.head()

In [ ]:
df["temp_mean_6h"] = df["temperature_c"].rolling(window=12, min_periods=1).mean()
df.head()

In [ ]:
df["humidity_mean_6h"] = df["humidity_pct"].rolling(window=12, min_periods=1).mean()
df.head()

## Normalizing

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Keep an unscaled copy of the temperature targets so outputs stay in Celsius
# while inputs (including the observed temperature) are standardized
target_col = 'target_temperature_c'
df[target_col] = df['temperature_c']

feature_cols = [col for col in df.select_dtypes(include='number').columns if col != target_col]
feature_indices = [df.columns.get_loc(col) for col in feature_cols]
target_index = df.columns.get_loc(target_col)

scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])



**Note:** Unlike `microclimate_forecast_model.ipynb`, this notebook currently keeps the `temperature_c` column out of the `StandardScaler` so the network trains and predicts in real-world temperature units. In the microclimate notebook the line `combined_dataframe.drop(columns=target_column_names)` only removes the *future target* columns, so the current-temperature input *is* standardized along with the other features. If you want identical normalization here, create a separate unscaled copy of the target (e.g., `target_temperature_c`) and include `temperature_c` in the scaler's `feature_cols`.


In [ ]:
df.describe()

# Creating the Datasets used by the Model

## Define Input Window Size and Prediction Window Size
We want to use the previous 24 hours of data to predict the next 12 hours of temperatures.

In [ ]:
sample_size_in_hrs = 0.5 # we resampled to 30-min chunks
window_size_in_hrs = 24

# since each instance is a 30-min period, and we want a 24hr window
seq_length = int(window_size_in_hrs / sample_size_in_hrs)

## Train, Test Split
We use 80% of the data to train. Then the remaining 20% is then split into equal 10% segments. **No shuffling is done as the time series data needs to stay in chronological order**.

In [ ]:
from sklearn.model_selection import train_test_split

# First split: 80% and 20%
df_train, df_temp = train_test_split(df, test_size=0.2,
                                     random_state=42, shuffle=False)

# Second split: split the remaining 20% into two 10% parts
df_valid, df_test = train_test_split(df_temp, test_size=0.5,
                                     random_state=42, shuffle=False)

print(f"Train: {len(df_train)}")
print(f"Valid: {len(df_valid)}")
print(f"Test: {len(df_test)}")


In [ ]:
df_train.info()

In [ ]:
df_train.head()

## Converting DataFrames to Timeseries Datasets

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf

def split_inputs_and_targets(
    multivariable_series,
    ahead: int = 24,   # next 24 half-hour slots
):
    # Split into input window and target horizon
    input_sequence = multivariable_series[:, :-ahead, :]
    # Use tf.gather to select columns using feature_indices
    inputs = tf.gather(input_sequence, tf.constant(feature_indices), axis=-1) # shape: (batch, seq_length, num_features)
    targets = multivariable_series[:, -ahead:, target_index]   # shape: (batch, ahead)

    # Tell TensorFlow the exact shapes so RNN can unroll
    # batch dimension stays None, time dimension is fixed
    inputs.set_shape((None, seq_length, len(feature_cols)))
    targets.set_shape((None, ahead))

    return inputs, targets

In [ ]:
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    df_train.to_numpy(),
    targets=None,
    sequence_length=seq_length + 24,
    batch_size=32,
    shuffle=True,
    seed=42
).map(split_inputs_and_targets)

train_ds

In [ ]:
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    df_valid.to_numpy(),
    targets=None,
    sequence_length=seq_length + 24,
    batch_size=32
).map(split_inputs_and_targets)
valid_ds

# Build and Compile the Model

## I Should Use Keras Tuner here once i get the base model to run

## Definition

In [ ]:
num_features = len(feature_cols)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(seq_length, num_features)),
    tf.keras.layers.SimpleRNN(64, return_sequences=True,unroll=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(64,unroll=True),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(24)
])


In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=15,
    restore_best_weights=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=optimizer,
    metrics=["mae"]
)

## Training

In [ ]:
history = model.fit(
    train_ds, validation_data=valid_ds,
    epochs=500, callbacks=[early_stopping_cb]
)

In [ ]:
history = model.fit(
    train_ds, validation_data=valid_ds,
    epochs=500, callbacks=[early_stopping_cb]
)

In [ ]:

import numpy as np

# Ensure the trained model has only finite weights before exporting to TFLite/X-CUBE-AI
for idx, weight in enumerate(model.get_weights()):
    if not np.all(np.isfinite(weight)):
        raise ValueError(f"Non-finite values found in weight tensor {idx}; check preprocessing and training stability before export.")
print("Model weights verified as finite.")


# Using the Model to Make Predictions

In [ ]:
def predict_12_hours(model, input_df, seq_length, start_idx, target_col_name,
                     window_size_in_hrs):
  X = input_df[feature_cols].to_numpy()[np.newaxis, start_idx:start_idx+seq_length]
  print(f"Shape of Input Data: {X.shape}")

  Y_pred = model.predict(X)[0]
  print(f"Model Prediction: {Y_pred}")

  actual_next_12_hrs = input_df[start_idx+seq_length:
                                start_idx+seq_length+window_size_in_hrs][target_col_name].to_numpy()
  print(f"Actual Next 12 Hours: {actual_next_12_hrs}")

  for i in range(window_size_in_hrs):
    model_pred_in_deg = Y_pred[i]
    actual_in_deg = actual_next_12_hrs[i]
    pred_error = model_pred_in_deg - actual_in_deg

    print(f"{(i+1) * 30} MINUTES INTO THE FUTURE!")
    print(f"Model Prediction: {model_pred_in_deg} degrees celcius.")
    print(f"Actual Value: {actual_in_deg} degrees celcius.")
    print(f"Prediction Error: {pred_error} degrees celcius.")
    print('*'*20)

  return Y_pred, actual_next_12_hrs


In [ ]:
Y_pred_deg, actual_val_deg = predict_12_hours(
    model, df_valid, seq_length, 0, target_col,
    window_size_in_hrs
)


# Evaluating the model on the test set

In [ ]:
df_test.info()

In [ ]:
test_ds = tf.keras.utils.timeseries_dataset_from_array(
    df_test.to_numpy(),
    targets=None,
    sequence_length=seq_length + 24,  # same as in training
    batch_size=32
).map(split_inputs_and_targets)


In [ ]:
results = model.evaluate(test_ds)

In [ ]:
test_mae = results[1]
test_mae

In [ ]:
test_mae_deg = test_mae
print(f"Test MAE Degrees Celcius: {test_mae_deg}")

# Preparing the Model for Deployment on the Embedded Board

## Structured pruning and footprint analysis
We apply a manual, structured pruning pass to the RNN (removing the weakest neurons per layer) and log the footprint at each stage (full, pruned, and quantized).
Pinned NumPy wheels remain to avoid ABI conflicts seen on Colab.


In [ ]:
import os
import tempfile
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2


def calculate_macs(model):
    sample_spec = tf.TensorSpec([1, seq_length, num_features], tf.float32)
    concrete_func = tf.function(model).get_concrete_function(sample_spec)
    frozen_func = convert_variables_to_constants_v2(concrete_func)
    graph_def = frozen_func.graph.as_graph_def()

    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd='op', options=opts)

    macs = flops.total_float_ops // 2 if flops is not None else None
    return macs


def get_model_size_kb(model):
    fd, temp_path = tempfile.mkstemp(suffix='.h5')
    os.close(fd)
    model.save(temp_path, include_optimizer=False)
    size_kb = os.path.getsize(temp_path) / 1024
    os.remove(temp_path)
    return size_kb


def summarize_keras_model(label, model):
    macs = calculate_macs(model)
    macs_display = f"{macs:,}" if macs is not None else 'N/A'
    params = model.count_params()
    size_kb = get_model_size_kb(model)
    print(f"""{label}:
 - Parameters: {params:,}
 - MACs (approx): {macs_display}
 - Size: {size_kb:.2f} KB
""")


def summarize_tflite_model(label, tflite_bytes, reference_keras_model=None):
    params = reference_keras_model.count_params() if reference_keras_model else 'N/A'
    macs = calculate_macs(reference_keras_model) if reference_keras_model else None
    macs_display = f"{macs:,}" if macs is not None else 'N/A'
    size_kb = len(tflite_bytes) / 1024
    print(f"""{label}:
 - Parameters: {params:,}
 - MACs (approx): {macs_display}
 - Size: {size_kb:.2f} KB
""")


In [ ]:
# Baseline footprint before pruning
summarize_keras_model("Full Keras model", model)

In [ ]:
pruned_model, pruning_masks = structured_prune_model(model, target_sparsity=0.5)

# Use a fresh optimizer so it can track the cloned model's variables.
# Reusing the previously built optimizer causes KeyErrors when it sees
# variables (e.g., the SimpleRNN kernel) that were not part of the original
# optimizer state.
pruning_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
pruning_optimizer.build(pruned_model.trainable_variables)

pruned_model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=pruning_optimizer,
    metrics=['mae']
)

pruning_callbacks = [
    StructuredPruningCallback(pruning_masks),
    early_stopping_cb,
]

pruned_history = pruned_model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=10,
    callbacks=pruning_callbacks,
)

summarize_keras_model("Structured pruned model", pruned_model)


## Quantizing, Pruning and Saving the TFLite Model

In [ ]:
def representative_dataset():
    for batch in tf.keras.utils.timeseries_dataset_from_array(
        data=df_train[feature_cols].to_numpy().astype(np.float32),
        targets=None,
        sequence_length=seq_length,
        batch_size=1,
    ).take(200):
        yield [batch]

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]
# Disable lowering of tensor list ops to avoid TF Lite conversion failures for RNN layers.
# converter._experimental_lower_tensor_list_ops = False

# Keep strict int8 inputs/outputs for fully quantized inference.
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

quantized_model = converter.convert()
with open("quantized_rnn_model.tflite", "wb") as f:
    f.write(quantized_model)


summarize_tflite_model("Quantized int8 model", quantized_model, reference_keras_model=pruned_model)

## Reload the TFLite Model

In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="quantized_rnn_model.tflite")
interpreter.allocate_tensors()

# Get input and output details (optional)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model loaded successfully!")
print("Input details:", input_details)
print("Output details:", output_details)

In [ ]:

# Evaluating the size of the tflite model
import os

file_path = "quantized_rnn_model.tflite"
size_in_bytes = os.path.getsize(file_path)
size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024

print(f"Model size: {size_in_bytes} bytes ({size_in_kb:.2f} KB / {size_in_mb:.2f} MB)")


## Using the TFLite Model to Make a Prediction

In [ ]:
import numpy as np

# Prepare input (same as Keras)
input_scale, input_zero_point = input_details[0]["quantization"]
output_scale, output_zero_point = output_details[0]["quantization"]

def quantize_input(window: np.ndarray) -> np.ndarray:
    return np.clip(np.round(window / input_scale + input_zero_point), -128, 127).astype(np.int8)

def dequantize_output(tensor: np.ndarray) -> np.ndarray:
    return (tensor.astype(np.float32) - output_zero_point) * output_scale

X = df_valid[feature_cols].to_numpy()[np.newaxis, :seq_length].astype(np.float32)  # shape: (1, seq_length, features)
X_int8 = quantize_input(X) # Quantize input

# Set input tensor
interpreter.set_tensor(input_details[0]["index"], X_int8) # Pass quantized int8 input

# Run inference
interpreter.invoke()

# Get prediction
output_data_int8 = interpreter.get_tensor(output_details[0]["index"])
output_data = dequantize_output(output_data_int8) # Dequantize output
print("Prediction:", output_data)


## Evaluating the TFLite Model on the Test Dataset

In [ ]:
import numpy as np

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get quantization parameters from the loaded model details
input_scale, input_zero_point = input_details[0]["quantization"]
output_scale, output_zero_point = output_details[0]["quantization"]

def quantize_input(window: np.ndarray) -> np.ndarray:
    return np.clip(np.round(window / input_scale + input_zero_point), -128, 127).astype(np.int8)

def dequantize_output(tensor: np.ndarray) -> np.ndarray:
    return (tensor.astype(np.float32) - output_zero_point) * output_scale

seq_length = 48  # or whatever your window size is
num_features = len(feature_cols)

# Store predictions and true values
preds = []
trues = []

for start in range(len(df_test) - seq_length - 24 + 1):  # 24 is your prediction horizon
    X = df_test[feature_cols].to_numpy()[start:start+seq_length].astype(np.float32)
    X_int8 = quantize_input(X[np.newaxis, ...])  # Quantize input

    interpreter.set_tensor(input_details[0]['index'], X_int8) # Pass quantized int8 input
    interpreter.invoke()

    output_data_int8 = interpreter.get_tensor(output_details[0]['index'])
    y_pred = dequantize_output(output_data_int8)[0] # Dequantize output
    preds.append(y_pred)

    # True values for the next 24 steps of the target column (e.g., temperature)
    y_true = df_test.iloc[start+seq_length:start+seq_length+24][target_col].to_numpy()
    trues.append(y_true)


In [ ]:
preds = np.array(preds)
trues = np.array(trues)

# Temperature targets are kept in their original scale, so predictions are already in Celsius
mae = np.mean(np.abs(preds - trues))


In [ ]:
print(f"TFLite Model Test MAE: {mae}")
print(f"TFLite Model Test MAE Degrees Celcius: {mae}")